In [ ]:
from numpy import asarray

In [ ]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataRegressor

In [ ]:
import pandas as pd

In [ ]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [ ]:
import tensorflow as tf

In [ ]:
import autokeras as ak

In [ ]:
raw_dataset = pd.read_csv("features.csv", na_values='?'
                          , comment='\t'
                          , skipinitialspace=True)

In [ ]:
raw_dataset

In [ ]:
dataset = raw_dataset.copy()

In [ ]:
def train_cats(df):
	for n,c in df.items():
	    if is_string_dtype(c): 
	        df[n] = c.astype('category').cat.as_ordered()

In [ ]:
train_cats(dataset)

In [ ]:
dataset.drop(['kost_name_rough'], axis=1, inplace=True)

In [ ]:
dataset

In [ ]:
max_n_cat = None

for n,c in dataset.items(): 
    if not is_numeric_dtype(c) and (max_n_cat is None or c.nunique()>max_n_cat):
        dataset[n] = c.cat.codes+1

In [ ]:
dataset

In [ ]:
print(list(dataset.groupby(["kota"]).size().reset_index(name='counts')["kota"]))
print(list(raw_dataset.groupby(["kota"]).size().reset_index(name='counts')["kota"]))

print(list(dataset.groupby(["type_kos"]).size().reset_index(name='counts')["type_kos"]))
print(list(raw_dataset.groupby(["type_kos"]).size().reset_index(name='counts')["type_kos"]))

print(list(dataset.groupby(["area"]).size().reset_index(name='counts')["area"]))
print(list(raw_dataset.groupby(["area"]).size().reset_index(name='counts')["area"]))

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
dataset = dataset.values
dataset = dataset.astype('float32')
X, y = dataset[:, :-1], dataset[:, -1]
print(X.shape, y.shape)

In [ ]:
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
search = StructuredDataRegressor(loss='mean_absolute_error',directory='models_saved_data/')

In [ ]:
# perform the search
search.fit(x=X_train, y=y_train, verbose=0)

In [ ]:
# evaluate the model
mae, _ = search.evaluate(X_test, y_test, verbose=0)
print('MAE: %.3f' % mae)

In [ ]:
X_new = asarray([[5,1,56,6]]).astype('float32')
yhat = search.predict(X_new)
print('Predicted: %.3f' % yhat[0])

In [ ]:
model = search.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file

In [ ]:
model.save('getkos_nas', save_format="tf")

In [ ]:
reloaded = tf.keras.models.load_model('models_saved_data/structured_data_regressor/best_model', custom_objects=ak.CUSTOM_OBJECTS)

In [ ]:
X_new = asarray([[5,1,56,6]]).astype('float32')
yhat = reloaded.predict(X_new)
print('Predicted: %.3f' % yhat[0])

In [ ]:
reloaded.summary()

In [ ]:
test_result = reloaded.evaluate(
    X_test, y_test, verbose=0
)

test_result # 582997.250

In [ ]:
try:
    reloaded.save("model_autokeras.h5")
except:
    reloaded.save("model_autokeras", save_format="tf")

# from tensorflow.keras.models import load_model

In [ ]:
loaded_model = tf.keras.models.load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

X_new = asarray([[5,1,56,6]]).astype('float32')
yhat = loaded_model.predict(X_new)
print('Predicted: %.3f' % yhat[0])

In [ ]:
loaded_model.save('getkos_nas',save_format="tf")

## use another env myenv

In [ ]:
# !pip install tf-nightly

In [ ]:
# !pip install git+https://github.com/keras-team/keras-tuner.git
# !pip install autokeras

In [ ]:
import tensorflow as tf
import autokeras as ak
model=tf.keras.models.load_model("models_saved_data/structured_data_regressor/best_model", custom_objects=ak.CUSTOM_OBJECTS)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
# converter = tf.lite.TFLiteConverter.from_keras_model("model_autokeras.h5")
# converter.experimental_new_converter = True
# tflite_model = converter.convert()